<a href="https://colab.research.google.com/github/hdim91/RA---KDI/blob/master/1st_%EC%9D%B4%EB%9E%9C%EB%93%9C(%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup as bs
import requests,re,codecs
import pandas as pd
import csv
import time

In [0]:
# Searchpad

# div class="deSearchBox scrollbarStyle mCustomScrollbar _mCS_12" id="deSearchBox" tyle="display: block;"
# div class="deSeTd tdA"
#     input type="text" clas="dateBox" id="fromdate"
#     들어가야 할 것: 2006-01-01
#     input type="text" clas="dateBox" id="todate"
#     들어가야 할 것: 2008-12-31

# div class "deSearchBtn"
#     button id = "searchSubmit" class="activate"

In [0]:
# Query data for searching for "이랜드"
lines = '''pageInfo:bksMain
login_chk:null
LOGIN_SN:null
LOGIN_NAME:null
indexName:news
keyword:근로시간
byLine:
searchScope:1
searchFtr:1
startDate:2018-01-01
endDate:2019-04-09
sortMethod:date
contentLength:100
providerCode:
categoryCode:
incidentCode:
dateCode:
highlighting:true
sessionUSID:
sessionUUID:test
listMode:
categoryTab:
newsId:
delnewsId:
delquotationId:
delquotationtxt:
filterProviderCode:
filterCategoryCode:
filterIncidentCode:
filterDateCode:
filterAnalysisCode:
startNo:1
resultNumber:100
topmenuoff:
resultState:
keywordJson:{"searchDetailTxt1":"근로시간","agreeDetailTxt1":"","needDetailTxt1":"","exceptDetailTxt1":"","o_id":"option1","startDate":"2018-01-01","endDate":"2019-04-09","providerNm":"","categoryNm":"","incidentCategoryNm":"","providerCode":"","categoryCode":"","incidentCategoryCode":"","searchFtr":"1","searchScope":"1","searchKeyword":"근로시간"}
keywordFilterJson:
realKeyword:
keywordYn:Y
totalCount:
interval:
quotationKeyword1:
quotationKeyword2:
quotationKeyword3:
printingPage:
searchFromUseYN:N
searchFormName:
searchFormSaveSn:
mainTodayPersonYn:
period:'''.splitlines()

In [0]:
# Construct a dictionary for the targeting url
data = {}
for line in lines:
    key,value = line.split(':',1)
    if value=='null':
        value = None
    data[key] = value

In [0]:
# search for html
response = requests.post(result_url,data=data)
html = response.text
soup = bs(html,"html.parser")

In [0]:
url_list = {}
result_url = "https://www.bigkinds.or.kr/news/detailSearch.do"

for index in range(1,100): # There are 51 of indexes for the keyword "이랜드"
    data['startNo'] = index # Set the page number
    response = requests.post(result_url,data=data) # Call url of targeting page
    html = response.text # Save html text as "html"
    soup = bs(html,"html.parser") # Parsing html text
    for tag in soup.select('.resultList li h3'):
        key = tag.text.strip() # Setting title of an article as the key value of dictionary
        doc_id = tag['id'].replace('news_','')
        value = 'https://www.bigkinds.or.kr/news/detailView.do?docId={}&returnCnt=1'.format(doc_id) # Setting article url as the value of dictionary
        url_list[key] = value # Construct the dictionary of the list of news articles' titles and urls 
    

In [0]:
main_df = pd.DataFrame(list(url_list.items()),columns=['Title','url'])

In [11]:
main_df.shape

(9369, 2)

In [13]:
main_df.to_excel("https://drive.google.com/drive/folders/1n3_xMUnvFLzg82Hjd_zNm009RPK0NwCP/url_df.xlsx") # Save url_df as excel file for future use - just in case

FileNotFoundError: ignored

In [0]:
articles = {}
date_dict = {}
provider_dict = {}

for url in main_df['url']:
    try:
        response = requests.post(url)
        html = response.text
        dict1 = eval(html)
        content = dict1['detail']['CONTENT']
        articles[url] = content
        date=dict1['detail']['DATE']
        date_dict[url]=date
        provider=dict1['detail']['PROVIDER']
        provider_dict[url]=provider
    except:
        pass

articles_df = pd.DataFrame(list(articles.items()),columns=['url','Contents'])
total_df = pd.merge(main_df,articles_df,on="url")
date_df = pd.DataFrame(list(date_dict.items()),columns=['url','Date'])
total_df = pd.merge(total_df,date_df,on='url')
provider_df = pd.DataFrame(list(provider_dict.items()),columns=['url','Provider'])
total_df = pd.merge(total_df,provider_df,on='url')

In [0]:
print(total_df['Provider'].unique()) # number of press discussed issues about "이랜드"

['전북도민일보' '파이낸셜뉴스' '영남일보' '아시아경제' '경남도민일보' '경상일보' '매일경제' '전자신문' '매일신문'
 '한겨레' '국민일보' 'MBC' 'SBS' '내일신문' '한국경제' '세계일보' '경향신문' '광주일보' '서울경제' '문화일보'
 '국제신문' '한국일보' '무등일보' '서울신문' '머니투데이' '광주매일신문' '경기일보' '중부일보' '대구일보' '디지털타임스'
 '한라일보' '중도일보' '충북일보' '충청투데이' '강원도민일보' '대전일보' '경인일보' '부산일보' '중부매일' '전남일보'
 '전북일보' '경남신문' '헤럴드경제' '제민일보']


In [0]:
total_df.to_excel("News.xlsx") # Export the data as an excel file